In [1]:
%pip install -q --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [1]:
#using this cell to import the necessary libraries for the notebook
import pandas as pd

In [2]:
# setting pandas options to display more columns and rows in the output
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 110530)

# setting pandas options to expand the DataFrame representation
pd.set_option('display.expand_frame_repr', True)

# use this to reset all options to default
# pd.reset_option(`all`)

In [3]:
# reading the CSV file into a pandas Dataframe
df = pd.read_csv('./Insurance.csv')

# peaking at the df to have an overview of the data
df.head()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
# didplaying shape to peak at dimensions
df.shape

(1338, 7)

In [5]:
# displaying the info of the df to understand data types and non-null counts
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [6]:
# cheching for missing values and handling them
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

There is no null values

In [7]:
# Checking for duplicate rows
df.duplicated().sum()

1

Here we also found one duplicated row and droped it

In [10]:
print(df[df.duplicated(keep=False)])  
# Check and show only duplicates in the entire DataFrame where if you don't use df[] it will return a df with all rows 
# labeled true and false
# This happens because df[condition] returns only the seleted rows that meat the condition

     age   sex    bmi  children smoker     region    charges
195   19  male  30.59         0     no  northwest  1639.5631
581   19  male  30.59         0     no  northwest  1639.5631


In [11]:
# removing duplicate rows and resetting the index to avoid index issues 
# after dropping duplicates the index is not automatically reset so we need to do it manually
# the cleanest way to do this is to chain methods and reassign df = df.drop_duplicates().reset_index(drop=True)
# you can not chain and use inplace = True at the same time because it returns null and you can't use methods with null
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [12]:
# getting a statistical summary of the df
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1337.0,39.222139,14.044333,18.0000,27.000,39.0000,51.00000,64.00000
bmi,1337.0,30.663452,6.100468,15.9600,26.290,30.4000,34.70000,53.13000
children,1337.0,1.095737,1.205571,0.0000,0.000,1.0000,2.00000,5.00000
charges,1337.0,13279.121487,12110.359656,1121.8739,4746.344,9386.1613,16657.71745,63770.42801


In [13]:
# getting the correlation matrix of the numeric columns in the df to understand relationships between them
# using numeric_only=True to avoid errors with non-numeric columns
df.corr(numeric_only=True)

,age,bmi,children,charges
age,1.000000,0.109344,0.041536,0.298308
bmi,0.109344,1.000000,0.012755,0.198401
children,0.041536,0.012755,1.000000,0.067389
charges,0.298308,0.198401,0.067389,1.000000


in here we see no values approaching 1 meaning no need to drop any of these features but we can also encode the remaining features and then redo the correlation matrix

In [14]:
df.nunique()

age           47
sex            2
bmi          548
children       6
smoker         2
region         4
charges     1337
dtype: int64

here we can see that sex, and smoker features can be encoded to 0 and 1, with the remaining region feature we can use one-hot encoder

In [15]:
# displaying the unique values in each column to understand the data better
for column in df.columns:
    print(f"{column}: {df[column].unique()[:10]}")  # Displaying only the first 10 unique values for brevity

age: [19 18 28 33 32 31 46 37 60 25]
sex: ['female' 'male']
bmi: [27.9   33.77  33.    22.705 28.88  25.74  33.44  27.74  29.83  25.84 ]
children: [0 1 3 2 5 4]
smoker: ['yes' 'no']
region: ['southwest' 'southeast' 'northwest' 'northeast']
charges: [16884.924    1725.5523   4449.462   21984.47061  3866.8552   3756.6216
  8240.5896   7281.5056   6406.4107  28923.13692]


In [17]:
# encoding categorical variables (sex and smoker) using one-hot encoding
# when using inplce=True with replce it wil raise a warning because it is trying to set a vlue on a copy of the column not afecting the original df
# to avoid this warning we can use the .replace() method without inplace=True and reassign the column
# I used the .astype(int) to void the silent conversion when using .replce() and avoid the warning and future proofing the code
df['sex'] = df['sex'].replace({'female': 0, 'male': 1}).astype(int)
df['smoker'] = df['smoker'].replace({'no': 0, 'yes': 1}).astype(int)

In [18]:
# peaaking at the df again to see the changes
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [19]:
# now encoding the region column using one-hot encoding
# One-hot encodes the column region.
# Adds the resulting dummy/indicator variables to the DataFrame.
# Drops the original region column.
# Keeps all new columns except the first category, to avoid multicollinearity.
df = pd.get_dummies(df, columns=['region'], drop_first=True)

# for df with mny objects to encode we can use the following code
# df = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns, drop_first=True) or 

# object_columns = df.select_dtypes(include=['object']).columns
# from category_encoders import OneHotEncoder, BinaryEncoder
# encoder = OneHotEncoder(cols=object_columns)
# df = encoder.fit_transform(df)

In [20]:
df.head()

,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,False,False,True
1,18,1,33.770,1,0,1725.55230,False,True,False
2,28,1,33.000,3,0,4449.46200,False,True,False
3,33,1,22.705,0,0,21984.47061,True,False,False
4,32,1,28.880,0,0,3866.85520,True,False,False


In [21]:
# Creating new features based on existing data
# 1. Age Group: Categorizing age into groups
# 2. BMI Category: Categorizing BMI into underweight, normal, overweight, and obese
# 3. Charges per Child: Calculating charges per child

# Defining age groups
age_bins = [0, 18, 35, 50, 65, 100]
age_labels = ['Child', 'Young Adult', 'Adult', 'Middle Aged', 'Senior']

# Defining BMI categories
bmi_bins = [0, 18.5, 24.9, 29.9, 40]
bmi_labels = ['Underweight', 'Normal', 'Overweight', 'Obese']

# Creating new features

df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)
df['bmi_category'] = pd.cut(df['bmi'], bins=bmi_bins, labels=bmi_labels)
df['charges_per_child'] = df['charges'] / (df['children'] + 1)  # Adding 1 to avoid division by zero

# Displaying the head of the updated dataframe to show the new features
df.head()

,age,sex,bmi,children,smoker,charges,region_northwest,region_southeast,region_southwest,age_group,bmi_category,charges_per_child
0,19,0,27.900,0,1,16884.92400,False,False,True,Young Adult,Overweight,16884.92400
1,18,1,33.770,1,0,1725.55230,False,True,False,Child,Obese,862.77615
2,28,1,33.000,3,0,4449.46200,False,True,False,Young Adult,Obese,1112.36550
3,33,1,22.705,0,0,21984.47061,True,False,False,Young Adult,Normal,21984.47061
4,32,1,28.880,0,0,3866.85520,True,False,False,Young Adult,Overweight,3866.85520


after we generated new features droping the original features comes after EDA or depending on what model we will be using for ml